<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
- Alumno: Federico M. Zoya

## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset La Biblia descargada en formato texto plano de https://archive.org/stream/la-biblia-latinoamericana_202308/La%20Biblia%20Latinoamericana_djvu.txt.

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('La Biblia.txt', sep='/n', header=None, engine='python')
df.head()

,0
0,¡Jesús ha resucitado!
1,"Ustedes que abren la Biblia, busquen a Jesús. ..."
2,"para rezar, o para instrucción nuestra, La Bib..."
3,la vida.
4,En el centro de la Biblia está la Cruz de Jesú...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 178768


### 1 - Preprocesamiento

In [5]:
#from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(str(row[0]))) #Se fuerza conversión a string debido a errores obtenidos por datos interpretados como flontantes

2025-05-22 15:26:02.457156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747938362.645187   36427 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747938362.698867   36427 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747938363.155330   36427 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747938363.155361   36427 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747938363.155363   36427 computation_placer.cc:177] computation placer alr

- Se parsea el texto, es decir, se procesa cada documento que conforma el corpus, y se separan en palabras que luego serán vectorizadas.

In [13]:
# Demos un vistazo
sentence_tokens[:3]

[['¡jesús', 'ha', 'resucitado'],
 ['ustedes',
  'que',
  'abren',
  'la',
  'biblia',
  'busquen',
  'a',
  'jesús',
  'la',
  'biblia',
  'no',
  'es',
  'un',
  'libro',
  'solamente'],
 ['para',
  'rezar',
  'o',
  'para',
  'instrucción',
  'nuestra',
  'la',
  'biblia',
  'es',
  'palabra',
  'de',
  'dios',
  'para',
  'comunicarnos']]

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

- Se sobrecarga el método callback para visualizar el error por cada época.

In [15]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     #vector_size=300,       # dimensionalidad de los vectores 
                     #vector_size=15,       # Como el vocabulario ronda las 14 mil palabras => Dim{Embeddings} = Raiz Cuarta de 14 mil => Aprox 11. Se define 15 a modo de prueba.
                     vector_size=100, 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

- Se instancia el vectorizados con los parámetros requeridos, se efectúan pruebas definiendo distintos largos de embeddings para corroborar resultados. En este sentido se verifica que la dimensión del vector afecta el resultado de la vectorización, y por consecuencia, del resultado de las operaciones vectoriales relacionadas a las mètricas de interes.

In [16]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

- A partir de la instancia del modelo y los términos del corpus, se genera el vocabulario.

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 178768


In [16]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 14423


### 3 - Entrenar embeddings

In [17]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 6722672.0
Loss after epoch 1: 5065860.0
Loss after epoch 2: 4909357.0
Loss after epoch 3: 4686247.0
Loss after epoch 4: 4598276.0
Loss after epoch 5: 4525242.0
Loss after epoch 6: 4520630.0
Loss after epoch 7: 4536752.0
Loss after epoch 8: 4471308.0
Loss after epoch 9: 4410196.0
Loss after epoch 10: 4366448.0
Loss after epoch 11: 4324472.0
Loss after epoch 12: 4295224.0
Loss after epoch 13: 4265420.0
Loss after epoch 14: 2048216.0
Loss after epoch 15: 964512.0
Loss after epoch 16: 951952.0
Loss after epoch 17: 937768.0
Loss after epoch 18: 928064.0
Loss after epoch 19: 922016.0


(17531623, 25882540)

- Se entrena el modelo para que genere un espacio de vectores denso a partir de los tokens generados con las palabras del vocabulario. Esto permite disponer de un espacio donde cada vector representa un concepto inferido a partir de lo aprendido en la etapa de entrenamiento. Es de esperar que la dirección de cada vector se correlacione semánticamente con los conceptos que el vector representa.

### 4 - Ensayar

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["jesús"], topn=10)

[('pablo', 0.6132212281227112),
 ('pedro', 0.6110264658927917),
 ('pilato', 0.5882518291473389),
 ('juan', 0.5843722224235535),
 ('ezequiel', 0.5701648592948914),
 ('vic', 0.5612123608589172),
 ('endemoniado', 0.5575493574142456),
 ('lutero', 0.5504878163337708),
 ('resucitado', 0.5475106239318848),
 ('canso', 0.5423581004142761)]

- A partir de un nombre (jesús en este caso), se obtienen en el listado de palabras similares otros nombres propios mencionados en el corpus. Es en este caso la asociación semántica predominante resulta evidente en las primeras palabras del listado. Tambien aparecen otras palabras que no son nombres propios, pero que guardan relación menos directa con la palabra buscada.

In [19]:
# Palabras que MENOS se relacionan con...:
#w2v_model.wv.most_similar(negative=["biblia"], topn=10)
#w2v_model.wv.most_similar(negative=["jesús"], topn=10)
w2v_model.wv.most_similar(negative=["malo"], topn=10)

[('torres', 0.05908196046948433),
 ('for', 0.046074312180280685),
 ('maría', 0.0320778489112854),
 ('hebrón', 0.02323792688548565),
 ('ciudades', 0.019428109750151634),
 ('tiendas', 0.013767692260444164),
 ('segunda', 0.013088934123516083),
 ('reúne', 0.009724117815494537),
 ('587', 0.009182213805615902),
 ('reinó', 0.00846576876938343)]

- Listado de palabras que menos se parecen o que menos correlacionadas están, es decir vectores asociados a una mayor ortogonalidad.

In [21]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dar"], topn=10)

[('darle', 0.6985543966293335),
 ('darlos', 0.6460245847702026),
 ('darles', 0.6194551587104797),
 ('darme', 0.6106230020523071),
 ('intervenir', 0.6046268939971924),
 ('darse', 0.6012704968452454),
 ('dárselo', 0.5958026647567749),
 ('recuperar', 0.5894939303398132),
 ('darnos', 0.5840307474136353),
 ('lograr', 0.5818242430686951)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["bondad"], topn=5)

[('misericordia', 0.7392441034317017),
 ('ternura', 0.6442787647247314),
 ("a'la", 0.6435837149620056),
 ('lucidez', 0.6393976211547852),
 ('paciente', 0.6339818239212036)]

- Se verifica con diversas pruebas la relación existente entre las palabras elegidas para la prueba.

In [31]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["salamín"])

KeyError: "Key 'salamín' not present in vocabulary"

In [32]:
# el método `get_vector` permite obtener los vectores:
vector_dios= w2v_model.wv.get_vector("bondad")
print(vector_dios)

[ 0.27948487  0.5422419   0.01192401 -0.3974666   0.38953003 -0.48999876
  0.38978606  0.543595    0.31481418 -0.3885466   0.33346367 -0.42916143
 -0.2023289  -0.675008    0.07852904  0.12250926 -0.47225922 -0.42087248
  0.7170898  -1.211247    0.5119354   0.10889079  1.1489388   0.33706596
 -0.11894103  0.32769978  0.05467651 -0.2320251  -0.08386101 -0.01251764
  0.02802159  0.19031079  0.17165548 -0.89250994 -0.7527319   0.3350974
 -0.14664821  0.9127899  -0.18524483 -0.99529576  0.4881246  -0.21661639
  0.5143047   0.2038148   0.17546597 -0.22342023 -0.021809   -0.35668015
  0.13262922  0.18567744  0.01088025 -0.40845698 -0.32363385 -0.03253867
  0.00858937  0.01135063 -0.74698657 -0.20404924  0.02675049 -0.06095853
  0.06006893 -0.08676057  0.07409714 -0.1152029  -0.47674668  0.0422935
  0.13947846  0.25103992 -0.3871872   0.77008545  0.4083684  -0.03415367
  0.9184711   0.13794102  0.2647675   0.45352966 -0.01044037 -0.6211611
  0.0694292  -0.2806563   0.4804734  -0.31514102 -0.07

- Se visualizan los valores del embedding asociado al término "bondad".

In [33]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_dios)

[('bondad', 1.0000001192092896),
 ('misericordia', 0.7392441034317017),
 ('ternura', 0.6442787647247314),
 ("a'la", 0.6435837149620056),
 ('lucidez', 0.6393976211547852),
 ('paciente', 0.6339818239212036),
 ('fide', 0.6302697062492371),
 ('fideli', 0.6267864108085632),
 ('sálvanos', 0.6266052722930908),
 ('benevolencia', 0.6225915551185608)]

In [34]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["amor"], topn=10)

[('misericordia', 0.5888675451278687),
 ('designio', 0.5852081179618835),
 ('anhelo', 0.583711564540863),
 ('dinamismo', 0.5808125138282776),
 ('apasionado', 0.5769360661506653),
 ('tierno', 0.5712573528289795),
 ('infinita', 0.5691033005714417),
 ('fraterno', 0.5668566823005676),
 ('misericordioso', 0.5591129660606384),
 ('porvenir', 0.5586978793144226)]

### 5 - Visualizar agrupación de vectores

In [35]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [36]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "browser"

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab
print("Vecs:", type(vecs), vecs.shape if hasattr(vecs, "shape") else "No shape")
print("Labels:", type(labels), len(labels) if hasattr(labels, "__len__") else "No length")
print("NaNs in vecs:", np.isnan(vecs).any())

fig.show()

Vecs: <class 'numpy.ndarray'> (14423, 2)
Labels: <class 'numpy.ndarray'> 14423
NaNs in vecs: False


In [42]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

- Es posible verificar gráficamente las relaciones semánticas entre palabras por su cercanía en el espacio de embeddings.

### Conclusiones:

- En el presente trabajo se pudieron verificar conceptos presentados en clase a partir del procesamiento de un texto cualquiera, en este caso, uno correspondiente a La biblia. Mediante el paquete Gensim se implementó un modelo word2vec para la vectorización de los tokens generados a partir del corpus procesado. Una vez entrenado el modelo y generado el espacio de embeddings, se procedió a verificar las métricas eligiendo palabras al azar del corpus y comprobando la similitud con otras palabras generando listados top10, tanto para las diez mas parecidas como para las menos parecidas. Por último se generaron representaciones gráficas en 2D y 3D del espacio generado utilizando T-SNE, con el objetivo de visualizar un manifold representativo de dicho espacio.
